## Introduction



  We&rsquo;re thinking about the problem of finding the cheapest possible
  nutritious diet.  Last time we argued that this problem could be
  expressed as a *linear program*
$$
    \min_x p'x
$$
such that
$$
   \begin{bmatrix}
      A\\
      -A
   \end{bmatrix}x \geq \begin{bmatrix}
                        b_{min}\\
                        -b_{max}
                      \end{bmatrix},
$$
  where $p$ is a vector of prices, $A$ is a matrix that maps
  vectors of quantities of food into vectors of nutrients, and where
  $b_{min}$ and $b_{max}$ are respectively dietary minimums
  and maximums of different nutrients.  We will sometimes stack the
  last, obtaining
  $$
      b = \begin{bmatrix}
                        b_{min}\\
                        -b_{max}
                      \end{bmatrix}.
  $$

Our job in this notebook: Specify the objects required by the linear
program $(p,A,b)$, then have the computer solve the problem for us.



## Setup



We need some particular versions of the following modules;



In [1]:
!pip install -r requirements.txt

## USDA Food Central DataBase



The USDA maintains a database of nutritional information, where
  different kinds of food are identified by an FDC number.  They do
  not provide any data on prices.  

To look up nutritional information, use api provided by the USDA at
[https://fdc.nal.usda.gov/](https://fdc.nal.usda.gov/).   You should sign up for a
free api key (see directions on page), then add that key here in
place of &ldquo;DEMO<sub>KEY</sub>&rdquo;.



In [4]:
apikey = "DEMO_KEY"  # Replace with a real key!  "DEMO_KEY" will be slow...

### Looking up foods



I&rsquo;ve written a little module `fooddatacentral` with the methods

-   `search`
-   `nutrients`
-   `units`



### FDC Search



Here&rsquo;s a little code to help look up FDC codes for foods of
different descriptions.



In [5]:
import fooddatacentral as fdc

fdc.search(apikey,"crunchy peanut butter")

,fdcId,description,lowercaseDescription,dataType,gtinUpc,publishedDate,brandOwner,ingredients,allHighlightFields,score,foodNutrients
0,454532,CRUNCHY PEANUT BUTTER,crunchy peanut butter,Branded,011110714619,2019-04-01,The Kroger Co.,"ROASTED PEANUTS, SUGAR, CONTAINS 2% OR LESS OF...","<b>Ingredients</b>: ROASTED <em>PEANUTS</em>, ...",1057.6873,"[{'nutrientId': 1087, 'nutrientName': 'Calcium..."
1,454839,CRUNCHY PEANUT BUTTER,crunchy peanut butter,Branded,015400454759,2019-04-01,"Western Family Foods, Inc.","PEANUTS, DEXTROSE, HYDROGENATED VEGETABLE OIL ...","<b>Ingredients</b>: <em>PEANUTS</em>, DEXTROSE...",1057.6873,"[{'nutrientId': 1087, 'nutrientName': 'Calcium..."
2,455474,CRUNCHY PEANUT BUTTER,crunchy peanut butter,Branded,041268196135,2019-04-01,Hannaford Bros. Co.,"ROASTED PEANUTS, SUGAR CONTAINS 2% OR LESS OF:...","<b>Ingredients</b>: ROASTED <em>PEANUTS</em>, ...",1057.6873,"[{'nutrientId': 1087, 'nutrientName': 'Calcium..."
3,455481,CRUNCHY PEANUT BUTTER,crunchy peanut butter,Branded,041268198146,2019-04-01,Hannaford Bros. Co.,"ROASTED PEANUTS, SUGAR, CONTAINS 2% OR LESS OF...","<b>Ingredients</b>: ROASTED <em>PEANUTS</em>, ...",1057.6873,"[{'nutrientId': 1087, 'nutrientName': 'Calcium..."
4,456909,CRUNCHY PEANUT BUTTER,crunchy peanut butter,Branded,710069016200,2019-04-01,Cachere Distributing Inc.,"PEANUTS, SUGAR, HYDROGENATED VEGETABLE OILS (C...","<b>Ingredients</b>: <em>PEANUTS</em>, SUGAR, H...",1057.6873,"[{'nutrientId': 1087, 'nutrientName': 'Calcium..."
5,543056,CRUNCHY PEANUT BUTTER,crunchy peanut butter,Branded,04656700415,2019-04-01,Raley's,"PEANUTS, SUGAR, HYDROGENATED VEGETABLE OIL (RA...","<b>Ingredients</b>: <em>PEANUTS</em>, SUGAR, H...",1057.6873,"[{'nutrientId': 1087, 'nutrientName': 'Calcium..."
6,512844,CRUNCHY PEANUT BUTTER,crunchy peanut butter,Branded,021110210110,2019-04-01,"Marsh Supermarkets, LLC","PEANUTS, SUGAR, CONTAINS 2% OR LESS OF: FULLY ...","<b>Ingredients</b>: <em>PEANUTS</em>, SUGAR, C...",1057.6873,"[{'nutrientId': 1087, 'nutrientName': 'Calcium..."
7,521183,CRUNCHY PEANUT BUTTER,crunchy peanut butter,Branded,070980134019,2019-04-01,"Central Grocers Co-Op, Inc.","SELECTED ROASTED PEANUTS, SUGAR, HYDROGENATED ...",<b>Ingredients</b>: SELECTED ROASTED <em>PEANU...,1057.6873,"[{'nutrientId': 1087, 'nutrientName': 'Calcium..."
8,535574,CRUNCHY PEANUT BUTTER,crunchy peanut butter,Branded,817974020039,2019-04-01,BRANDLESS,ORGANIC PEANUTS AND SEA SALT.,<b>Ingredients</b>: ORGANIC <em>PEANUTS</em> A...,1057.6873,"[{'nutrientId': 1087, 'nutrientName': 'Calcium..."
9,381015,CRUNCHY PEANUT BUTTER,crunchy peanut butter,Branded,041497121823,2019-04-01,"Weis Markets, Inc.","ROASTED PEANUTS, SUGAR, CONTAINS LESS THAN 2% ...","<b>Ingredients</b>: ROASTED <em>PEANUTS</em>, ...",1057.6873,"[{'nutrientId': 1087, 'nutrientName': 'Calcium..."


### FDC Nutrients



Once we know the `fdc_id` of a particular food we can look up a
variety of information on it.  We start with nutrients



In [6]:
id =  455481   # Put an FDC ID HERE!
fdc.nutrients(apikey,fdc_id=id)

,Quantity,Units
"Fatty acids, total saturated",10.94,g
Energy,625.00,kcal
Protein,21.88,g
"Sodium, Na",469.00,mg
"Fiber, total dietary",6.20,g
"Carbohydrate, by difference",21.88,g
"Sugars, total including NLEA",9.38,g
"Fatty acids, total trans",0.00,g
Total lipid (fat),50.00,g
"Vitamin A, IU",0.00,IU


## Prices



Now, let&rsquo;s begin thinking about constructing the objects we need for
the linear program.  Start with specifying $p$, the vector of prices.  

Also note that some kinds of foods need to have unit weights (in
grams) supplied under &ldquo;Units&rdquo;; e.g., extra large eggs are taken to
each weigh 56g.  These conversions can also often be found on the USDA
FDC website.  Othertimes not&#x2014;I still need to weigh a crumpet.

Food is purchased in particular units (gallons, pounds, grams).  And
in some cases the natural units are things like donuts or eggs, in
which case we may need to define our  own units (see the example of
&ldquo;xl<sub>egg</sub>&rdquo; below).  New units can be added to the file [./Data/food_units.txt](./Data/food_units.txt).



### Example: My Shopping Trip



Here&rsquo;s an example of describing some different kinds of food, along with
data on food prices.  This is all just based on a trip I took to the
grocery store, except that I&rsquo;ve used the USDA database to look up FDC
numbers.  

| Food|Quantity|Units|Price|Date|Location|FDC|
|---|---|---|---|---|---|---|
| Milk, 2% fat|1|gallon|4.99|<span class="timestamp-wrapper"><span class="timestamp">[2019-09-14 Sat]</span></span>|Monterey Market, Berkeley|336075|
| Eggs, extra large|12|xl<sub>egg</sub>|3.59|<span class="timestamp-wrapper"><span class="timestamp">[2019-09-14 Sat]</span></span>|Monterey Market, Berkeley|747997|
| Crumpets|6|crumpet|3.19|<span class="timestamp-wrapper"><span class="timestamp">[2019-09-14 Sat]</span></span>|Monterey Market, Berkeley|547313|
| Bananas|1|pound|3.15|<span class="timestamp-wrapper"><span class="timestamp">[2019-09-14 Sat]</span></span>|Monterey Market, Berkeley|173944|
| Carrots, Organic|2|pound|2.29|<span class="timestamp-wrapper"><span class="timestamp">[2019-09-14 Sat]</span></span>|Monterey Market, Berkeley|170393|
| Cauliflower|2.51|pound|4.24|<span class="timestamp-wrapper"><span class="timestamp">[2019-09-14 Sat]</span></span>|Monterey Market, Berkeley|169986|
| Endive, Red|1.26|pound|6.27|<span class="timestamp-wrapper"><span class="timestamp">[2019-09-14 Sat]</span></span>|Monterey Market, Berkeley|168412|
| Figs, black mission|1|pound|4.98|<span class="timestamp-wrapper"><span class="timestamp">[2019-09-14 Sat]</span></span>|Monterey Market, Berkeley|438223|
| Leeks, Organic|1|pound|1.29|<span class="timestamp-wrapper"><span class="timestamp">[2019-09-14 Sat]</span></span>|Monterey Market, Berkeley|169246|
| Lettuce, Little Gem|1|pound|5.98|<span class="timestamp-wrapper"><span class="timestamp">[2019-09-14 Sat]</span></span>|Monterey Market, Berkeley|342618|
| Mushrooms, King Oyster|1|pound|12|<span class="timestamp-wrapper"><span class="timestamp">[2019-09-14 Sat]</span></span>|Monterey Market, Berkeley|342623|
| Onion, yellow|1|pound|0.39|<span class="timestamp-wrapper"><span class="timestamp">[2019-09-14 Sat]</span></span>|Monterey Market, Berkeley|170000|
| Orange juice|0.5|gallon|8.98|<span class="timestamp-wrapper"><span class="timestamp">[2019-09-14 Sat]</span></span>|Monterey Market, Berkeley|414575|
| Parsnip|1|pound|1.98|<span class="timestamp-wrapper"><span class="timestamp">[2019-09-14 Sat]</span></span>|Monterey Market, Berkeley|170417|
| Potato, marble mix|1|pound|2.59|<span class="timestamp-wrapper"><span class="timestamp">[2019-09-14 Sat]</span></span>|Monterey Market, Berkeley|170032|
| Rhubarb|1|pound|1.84|<span class="timestamp-wrapper"><span class="timestamp">[2019-09-14 Sat]</span></span>|Monterey Market, Berkeley|167758|
| Potato, russet|10|pound|2.98|<span class="timestamp-wrapper"><span class="timestamp">[2019-09-14 Sat]</span></span>|Monterey Market, Berkeley|170030|
| Squash, Zucchini|1|pound|1.49|<span class="timestamp-wrapper"><span class="timestamp">[2019-09-14 Sat]</span></span>|Monterey Market, Berkeley|169291|



### A Second Example: Villages in South India



Information on prices for different goods is found in a collection
  of `csv` files in [./Data](./Data).  You can generate additional files by
  using a spreadsheet and exporting to the appropriate file format,
  then putting that file in the [./Data>](./Data>)directory.  These files should
  have the same columns and format as the example above.

Here are some goods for which prices and quantities consumed were
recorded in a survey conducted by the International Crops Research
Institute of the Semi-Arid Tropics of several villages in South
India in the 1970s & early 1980s.



In [7]:
import pandas as pd

df = pd.read_csv("./Data/icrisat_foods.csv",   # Prices for food consumed in Indian ICRISAT villages
                 dtype={'Price':float,
                        'Quantity':float,
                        'FDC':int})  
df

,Food,Quantity,Units,Price,Date,Location,FDC
0,Chillies,36.0,oz,24.49,[2018-11-05 Monday],Amazon,173476
1,Coconut,1.0,lb,5.99,[2018-11-05 Monday],nuts.com,170169
2,Gur (jaggery),1.0,lb,3.63,[2018-11-05 Monday],Udupi Jaggery Powder,567640
3,Jowar/Sorghum (Local variety),1.0,lb,3.99,[2018-11-05 Monday],nuts.com,447350
4,Milk,1.0,gal,1.98,[2018-11-05 Monday],Target,336070
5,Onion,10.0,lbs,6.99,[2018-11-05 Monday],loblaws.ca,170000
6,Redgram dhal,2.0,lbs,3.88,[2019-10-15 Monday],jet.com,172472
7,Sugar,4.0,lbs,2.39,[2019-10-15 Monday],Target,169656
8,Tea,100.0,ml,3.69,[2018-11-05 Monday],Target,561207


### Another Example: Stigler's Foods



In his 1945 paper George Stigler constructed a subsistence diet
chosen from 14 different goods (see Table B in [Stigler 1945](https://www.jstor.org/stable/pdf/1231810.pdf)), with
prices reported for the years 1939 & 1944.  

I&rsquo;ve looked up more recent prices for these same goods, and recorded
these at
[https://docs.google.com/spreadsheets/d/1ObK5N_5aVXzVHE7ZXWBg0kQvPS3k1enRwsUjhytwh5A/](https://docs.google.com/spreadsheets/d/1ObK5N_5aVXzVHE7ZXWBg0kQvPS3k1enRwsUjhytwh5A/).  

The code below allows us to collect data on different kinds of food
with their prices from google spreadsheets.

In this case, we use a function from a module I&rsquo;ve written,
 `eep153_tools.read_sheets`, to read the price data for the
Stigler goods.



In [12]:
import pandas as pd
from eep153_tools import read_sheets

#### Need private keys from json file (we're authenticating using "service accounts")
#!gpg --batch --passphrase "SECRET PASSPHRASE" -d ../students-9093fa174318.json.gpg > ../students-9093fa174318.json
####

df = read_sheets("1ObK5N_5aVXzVHE7ZXWBg0kQvPS3k1enRwsUjhytwh5A",
                 sheet="Table B",
                 json_creds='students-9093fa174318.json')

df

,Food,Quantity,Units,Price,Date,Location,FDC
0,Wheat Flour,160,oz,6.41,[2019-10-15 Monday],Deep Premium,390092
1,Wheat Cereal,510,g,2.69,[2019-10-15 Monday],Target / Kellogs,391549
2,Corn Meal,24,oz,1.69,[2019-10-15 Monday],Target,363629
3,Rolled Oats,18,oz,2.99,[2019-10-15 Monday],Meijer,734348
4,Evaporated Milk,12,oz,0.94,[2019-10-15 Monday],Walmart,648546
5,Cabbage,1,lbs,0.62,[2019-10-15 Monday],USDA,169975
6,Potatoes,5,lbs,1.47,[2019-10-15 Monday],Walmart,576920
7,Spinach,10,oz,2.38,[2019-10-15 Monday],Walmart,168462
8,Sweet Potatoes,1,lbs,1.05,[2019-10-15 Monday],Walmart,600987
9,Navy Beans,1,lbs,1.49,[2019-10-15 Monday],Target,535776


### Units & Prices



Now, the prices we observe can be for lots of different quantities and
 units.  The FDC database basically wants everything in either hundreds
 of grams (hectograms) or hundreds of milliliters (deciliters).  

Sometimes this conversion is simple; if the price we observe is for
something that weighs two kilograms, that&rsquo;s just 20 hectograms.
Different systems of weights and volumes are also easy; a five pound
bag of flour is approximately 22.68 hectograms.  

Othertimes things are more complicated.  If you observe the price of a
dozen donuts, that needs to be converted to hectograms, for example.  

A function `units` in the [fdc](fooddatacentral.py) module accomplishes this conversion
for many different units, using the `python` [pint module](https://pint.readthedocs.io/en/latest/).  A file
[./Data/food<sub>units.txt</sub>](Data/food_units.txt) can be edited to deal with odd cases such as
donuts, using a format described in the `pint` [documentation](https://pint.readthedocs.io/en/latest/defining.html). 

Here&rsquo;s an example of the usage of `fooddatacentral.units`:



In [9]:
# Try your own quantities and units.
# If units are missing try adding to ./Data/food_units.txt

print(fdc.units(5,'lbs'))
print(fdc.units(1,'gallon'))
print(fdc.units(2,'tea_bag'))
print(fdc.units(12,'donut'))

22.679618500000004 hectogram
37.85411783999999 deciliter
0.06 hectogram
nan milliliter


Now, use the `units` function to convert all foods to either
 deciliters or hectograms, to match FDC database:



In [10]:
# Convert food quantities to FDC units
df['FDC Quantity'] = df[['Quantity','Units']].T.apply(lambda x : fdc.units(x['Quantity'],x['Units']))

# Now divide price by the FDC Quantity to get, e.g., price per hectoliter
df['FDC Price'] = df['Price']/df['FDC Quantity']

df.dropna(how='any') # Drop food with any missing data

# To use minimum price observed
Prices = df.groupby('Food')['FDC Price'].min()

Prices

Food
Beets               0.5489510328403452 / hectogram
Cabbage             0.1366866025546241 / hectogram
Corn Meal          0.24838748206162872 / hectogram
Evaporated Milk    0.27631270193837987 / hectogram
Liver (Pork)       0.26235009200000425 / hectogram
Milk (Whole)       0.07000559387490936 / deciliter
Navy Beans         0.32848877065546755 / hectogram
Potatoes           0.06481590508235399 / hectogram
Rolled Oats         0.5859397012735857 / hectogram
Spinach             0.8395202944000136 / hectogram
Sugar              0.13172620165546434 / hectogram
Sweet Potatoes     0.23148537529412144 / hectogram
Wheat Cereal        0.5274509803921568 / hectogram
Wheat Flour        0.14131631006050652 / hectogram
Name: FDC Price, dtype: object

## Mapping to Nutrients



Next we want to build the matrix $A$, which maps quantities of food
 into nutrients.  We have a list of foods with prices.  Do lookups on USDA database
 to get nutritional information.



In [13]:
import fooddatacentral as fdc

D = {}
count = 0
for food in  df.Food.tolist():
    try:
        FDC = df.loc[df.Food==food,:].FDC[count]
        count+=1
        D[food] = fdc.nutrients(apikey,FDC).Quantity
    except AttributeError: 
        warnings.warn("Couldn't find FDC Code %s for food %s." % (food,FDC))        

D = pd.DataFrame(D,dtype=float)

D

,Wheat Flour,Wheat Cereal,Corn Meal,Rolled Oats,Evaporated Milk,Cabbage,Potatoes,Spinach,Sweet Potatoes,Navy Beans,Sugar,Beets,Liver (Pork),Milk (Whole)
10:0,NaN,NaN,NaN,NaN,NaN,0.00,NaN,0.00,NaN,NaN,0.00,0.00,0.00,NaN
12:0,NaN,NaN,NaN,NaN,NaN,0.00,NaN,0.00,NaN,NaN,0.00,0.00,0.00,NaN
14:0,NaN,NaN,NaN,NaN,NaN,0.00,NaN,0.01,NaN,NaN,0.00,0.00,0.02,NaN
14:1,NaN,NaN,NaN,NaN,NaN,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15:0,NaN,NaN,NaN,NaN,NaN,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Vitamin K (Dihydrophylloquinone),NaN,NaN,NaN,NaN,NaN,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Vitamin K (phylloquinone),NaN,NaN,NaN,NaN,NaN,76.00,NaN,482.90,NaN,NaN,0.00,0.20,NaN,NaN
Vitamins and Other Components,NaN,NaN,NaN,NaN,NaN,0.00,NaN,0.00,NaN,NaN,0.00,0.00,0.00,NaN
Water,NaN,NaN,NaN,NaN,NaN,92.18,NaN,91.40,NaN,NaN,0.23,87.58,71.06,NaN


## Dietary Requirements



We&rsquo;ve figured out some foods we can buy, the nutritional content of
those foods, and  the price of the foods.  Now we need to say
something about nutritional requirements, and construct the vectors
$b_{min}$ and $b_{max}$.   Our data for this is based
on  US government recommendations available at
[https://www.dietaryguidelines.gov/sites/default/files/2019-05/2015-2020_Dietary_Guidelines.pdf](https://www.dietaryguidelines.gov/sites/default/files/2019-05/2015-2020_Dietary_Guidelines.pdf).
Note that we&rsquo;ve tweaked the nutrient labels to match those in the FDC
data.

We&rsquo;ve broken down the requirements into three different tables.  The
first is *minimum* quantities that we need to  satisfy.  For example,
this table tells us that a 20 year-old female needs at least 46 grams
of protein per day.



In [23]:
bmin = pd.read_csv('./diet_minimums.csv').set_index('Nutrition').iloc[:,2:]
bmin

,C 1-3,F 4-8,M 4-8,F 9-13,M 9-13,F 14-18,M 14-18,F 19-30,M 19-30,F 31-50,M 31-50,F 51+,M 51+
Nutrition,,,,,,,,,,,,,
Energy,1000.0,1200.0,1400.0,1600.0,1800.0,1800.0,2200.0,2000.0,2400.0,1800.0,2200.0,1600.0,2000.0
Protein,13.0,19.0,19.0,34.0,34.0,46.0,52.0,46.0,56.0,46.0,56.0,46.0,56.0
"Fiber, total dietary",14.0,16.8,19.6,22.4,25.2,25.2,30.8,28.0,33.6,25.2,30.8,22.4,28.0
"Folate, DFE",150.0,200.0,200.0,300.0,300.0,400.0,400.0,400.0,400.0,400.0,400.0,400.0,400.0
"Calcium, Ca",700.0,1000.0,1000.0,1300.0,1300.0,1300.0,1300.0,1000.0,1000.0,1000.0,1000.0,1200.0,1000.0
"Carbohydrate, by difference",130.0,130.0,130.0,130.0,130.0,130.0,130.0,130.0,130.0,130.0,130.0,130.0,130.0
"Iron, Fe",7.0,10.0,10.0,8.0,8.0,15.0,11.0,18.0,8.0,18.0,8.0,8.0,8.0
"Magnesium, Mg",80.0,130.0,130.0,240.0,240.0,360.0,410.0,310.0,400.0,320.0,420.0,320.0,420.0
Niacin,6.0,8.0,8.0,12.0,12.0,14.0,16.0,14.0,16.0,14.0,16.0,14.0,16.0


This next table specifies *maximum* quantities.  Our 20 year-old
female shouldn&rsquo;t have more than 2300 milligrams of sodium per day.



In [22]:
bmax = pd.read_csv('./diet_maximums.csv').set_index('Nutrition').iloc[:,2:]
bmax

,C 1-3,F 4-8,M 4-8,F 9-13,M 9-13,F 14-18,M 14-18,F 19-30,M 19-30,F 31-50,M 31-50,F 51+,M 51+
Nutrition,,,,,,,,,,,,,
"Sodium, Na",1500,1900,1900,2200,2200,2300,2300,2300,2300,2300,2300,2300,2300


In [16]:
# Choose sex/age group:
group = "F 19-30"

## Putting it together



Here we take the different pieces of the puzzle we&rsquo;ve developed and
put them together in the form of a linear program we can solve.
Recall that the mathematical problem we&rsquo;re trying to solve is
$$
    \min_x p'x
$$
such that
$$
     Ax \geq b
$$



### Objective function (c)



If we buy a bag of groceries with quantities given by $x$, the total
cost of the bag of groceries is the inner product of prices and
quantities.  Since we&rsquo;ve converted our units above, this gives us a
vector of prices where quantities are all in 100 g or ml units.



In [17]:
p = Prices.apply(lambda x:x.magnitude).dropna()

# Compile list that we have both prices and nutritional info for; drop if either missing
use = list(set(p.index.tolist()).intersection(D.columns.tolist()))
p = p[use]

p

Food
Potatoes           0.064816
Rolled Oats        0.585940
Corn Meal          0.248387
Sweet Potatoes     0.231485
Liver (Pork)       0.262350
Beets              0.548951
Milk (Whole)       0.070006
Cabbage            0.136687
Sugar              0.131726
Wheat Cereal       0.527451
Spinach            0.839520
Navy Beans         0.328489
Wheat Flour        0.141316
Evaporated Milk    0.276313
Name: FDC Price, dtype: float64

### Nutrient Mapping Matrix ($A$)



The matrix $A$ maps a bag of groceries $x$ into nutrients, but we
don&rsquo;t need to keep track of nutrients for which we don&rsquo;t have
contraints.



In [24]:
# Drop nutritional information for foods we don't know the price of,
# and replace missing nutrients with zeros.
Aall = D[p.index].fillna(0)

# Drop rows of A that we don't have constraints for.
Amin = Aall.loc[bmin.index]

Amax = Aall.loc[bmax.index]

# Minimum requirements involve multiplying constraint by -1 to make <=.
A = pd.concat([Amin,-Amax])

A

,Potatoes,Rolled Oats,Corn Meal,Sweet Potatoes,Liver (Pork),Beets,Milk (Whole),Cabbage,Sugar,Wheat Cereal,Spinach,Navy Beans,Wheat Flour,Evaporated Milk
Nutrition,,,,,,,,,,,,,,
Energy,74.00,350.00,364.000,85.00,561.000,180.000,62.00,103.000,1627.000,370.000,97.000,229.00,467.00,133.00
Protein,2.70,12.50,6.060,1.69,21.390,1.610,3.33,1.280,0.000,3.700,2.860,22.86,6.67,6.67
"Fiber, total dietary",2.00,10.00,3.000,3.40,0.000,2.800,0.00,2.500,0.000,3.700,2.200,25.70,6.70,0.00
"Folate, DFE",0.00,0.00,0.000,0.00,212.000,109.000,0.00,43.000,0.000,0.000,194.000,0.00,0.00,0.00
"Calcium, Ca",0.00,50.00,0.000,30.00,9.000,16.000,83.00,40.000,1.000,0.000,99.000,171.00,0.00,233.00
"Carbohydrate, by difference",16.89,67.50,81.820,20.34,2.470,9.560,5.00,5.800,99.770,88.890,3.630,60.00,66.67,10.00
"Iron, Fe",0.73,4.25,3.270,0.85,23.300,0.800,0.00,0.470,0.060,6.670,2.710,7.71,2.40,0.00
"Magnesium, Mg",0.00,0.00,0.000,0.00,18.000,23.000,0.00,12.000,0.000,59.000,79.000,0.00,0.00,0.00
Niacin,0.00,0.00,3.636,0.00,15.301,0.334,0.00,0.234,0.000,18.519,0.724,0.00,0.00,0.00


### Constraint vector ($b$)



Finally, the right hand side vector $b$ in the expression
$$
    Ax\geq b
$$



In [25]:
b = pd.concat([bmin,-bmax]) # Note sign change for max constraints

b

,C 1-3,F 4-8,M 4-8,F 9-13,M 9-13,F 14-18,M 14-18,F 19-30,M 19-30,F 31-50,M 31-50,F 51+,M 51+
Nutrition,,,,,,,,,,,,,
Energy,1000.0,1200.0,1400.0,1600.0,1800.0,1800.0,2200.0,2000.0,2400.0,1800.0,2200.0,1600.0,2000.0
Protein,13.0,19.0,19.0,34.0,34.0,46.0,52.0,46.0,56.0,46.0,56.0,46.0,56.0
"Fiber, total dietary",14.0,16.8,19.6,22.4,25.2,25.2,30.8,28.0,33.6,25.2,30.8,22.4,28.0
"Folate, DFE",150.0,200.0,200.0,300.0,300.0,400.0,400.0,400.0,400.0,400.0,400.0,400.0,400.0
"Calcium, Ca",700.0,1000.0,1000.0,1300.0,1300.0,1300.0,1300.0,1000.0,1000.0,1000.0,1000.0,1200.0,1000.0
"Carbohydrate, by difference",130.0,130.0,130.0,130.0,130.0,130.0,130.0,130.0,130.0,130.0,130.0,130.0,130.0
"Iron, Fe",7.0,10.0,10.0,8.0,8.0,15.0,11.0,18.0,8.0,18.0,8.0,8.0,8.0
"Magnesium, Mg",80.0,130.0,130.0,240.0,240.0,360.0,410.0,310.0,400.0,320.0,420.0,320.0,420.0
Niacin,6.0,8.0,8.0,12.0,12.0,14.0,16.0,14.0,16.0,14.0,16.0,14.0,16.0


## Solving the problem



First, we find a solution to the problem



In [26]:
from  scipy.optimize import linprog as lp
import numpy as np

tol = 1e-6 # Numbers in solution smaller than this (in absolute value) treated as zeros

# Now solve problem!  (Note that the linear program solver we'll use assumes
# "less-than-or-equal" constraints.  We can switch back and forth by
# multiplying $A$ and $b$ by $-1$.)

result = lp(p, -A, -b[group], method='interior-point')

result

     con: array([], dtype=float64)
     fun: 6.945643124529246
 message: 'Optimization terminated successfully.'
     nit: 17
   slack: array([3.55202804e+02, 1.23305333e+01, 1.90529903e-09, 1.25574972e+03,
       9.13007625e-09, 2.57213628e-09, 2.86631905e+01, 3.07934367e+02,
       1.00355487e+01, 7.05551884e-09, 6.97837095e-08, 3.52487451e+00,
       1.46984647e-11, 8.93904192e+03, 2.35655346e+01, 1.27387612e+00,
       1.81161694e+02, 1.71418435e-11, 3.50674819e+03, 2.70319451e+00,
       1.49034880e+03])
  status: 0
 success: True
       x: array([2.08703116e-09, 1.49089783e-11, 8.08417523e-11, 3.50125424e-11,
       9.52707375e-01, 1.33780339e-11, 2.10209472e+00, 7.07420363e-01,
       4.56979568e-01, 2.14953741e-01, 7.33689012e+00, 3.61671672e-01,
       4.33235807e-10, 6.11046219e-11])

Let&rsquo;s interpret this.  Start with the cost of the solution:



In [27]:
print("Cost of diet for %s is $%4.2f per day." % (group,result.fun))

Cost of diet for F 19-30 is $6.95 per day.


Next, what is it we&rsquo;re actually eating?



In [29]:
# Put back into nice series
diet = pd.Series(result.x,index=p.index)

print("\nYou'll be eating (in 100s of grams or milliliters):")
print(diet[diet >= tol])  # Drop items with quantities less than precision of calculation.


You'll be eating (in 100s of grams or milliliters):
Food
Liver (Pork)    0.952707
Milk (Whole)    2.102095
Cabbage         0.707420
Sugar           0.456980
Wheat Cereal    0.214954
Spinach         7.336890
Navy Beans      0.361672
dtype: float64


Given this diet, what are nutritional outcomes?



In [31]:
tab = pd.DataFrame({"Outcome":np.abs(A).dot(diet),"Recommendation":np.abs(b[group])})
print("\nWith the following nutritional outcomes of interest:")
tab


With the following nutritional outcomes of interest:


,Outcome,Recommendation
Nutrition,,
Energy,2355.202804,2000.0
Protein,58.330533,46.0
"Fiber, total dietary",28.000000,28.0
"Folate, DFE",1655.749723,400.0
"Calcium, Ca",1000.000000,1000.0
"Carbohydrate, by difference",130.000000,130.0
"Iron, Fe",46.663190,18.0
"Magnesium, Mg",617.934367,310.0
Niacin,24.035549,14.0


Finally, what are the constraints that bind?  Finding a less expensive
diet might involve finding less expensive sources for these particular nutrients.



In [32]:
print("\nConstraining nutrients are:")
excess = tab.diff(axis=1).iloc[:,1]
print(excess.loc[np.abs(excess) < tol].index.tolist())


Constraining nutrients are:
['Fiber, total dietary', 'Calcium, Ca', 'Carbohydrate, by difference', 'Phosphorus, P', 'Potassium, K', 'Thiamin', 'Vitamin E (alpha-tocopherol)']
